# name, gender 읽어오기 ETL 버전의 load 함수에 들어갈 내용 채워보기

1. INSERT INTO SQL 완성하기

In [ ]:
def load(csv_reader):
    cur = get_Redshift_connection()    
    for name, gender in csv_reader:
        sql = """
            INSERT INTO raw_data.name_gender
            VALUES (name, gender)
            """
        cur.execute(sql)

2. Full-Refresh형태로 동작하게 load 함수의 내용을 변경하기

In [ ]:
# BEGIN;END
def load(csv_reader):
    cur = get_Redshift_connection() 
    # cur: autocommit = True - BEGIN;END
    sql = """BEGIN;CREATE TALBE IF NOT EXISTS raw_data.name_gender (
            name varchar(64),
            gender varchar(16)
            );
            DELETE FROM raw_data.name_gender;"""
    cur.execute(sql)   
    for name, gender in csv_reader:
        sql = """
            INSERT INTO raw_data.name_gender
            VALUES ({name}, {gender});
            """.format(name=name, gender=gender)
        cur.execute(sql)
    cur.execute("END")

# try/catch. autocommit = TRUE
def load(csv_reader):
    cur = get_Redshift_connection() 
    try:
      sql = """BEGIN;CREATE TALBE IF NOT EXISTS raw_data.name_gender (
              name varchar(64),
              gender varchar(16)
              );
              DELETE FROM raw_data.name_gender;"""
      cur.execute(sql)   
      for name, gender in csv_reader:
          sql = """
              INSERT INTO raw_data.name_gender
              VALUES ({name}, {gender});
              """.format(name=name, gender=gender)
          cur.execute(sql)
      cur.execute("END") # cur.execute("COMMIT")
    except Exception as e:
      cur.execute("ROLLBACK")    

# try/catch. autocommit = FALSE
def load(csv_reader):
    cur = get_Redshift_connection() 
    try:
      sql = """CREATE TALBE IF NOT EXISTS raw_data.name_gender (
              name varchar(64),
              gender varchar(16)
              );
              DELETE FROM raw_data.name_gender;"""
      cur.execute(sql)   
      for name, gender in csv_reader:
          sql = """
              INSERT INTO raw_data.name_gender
              VALUES ({name}, {gender});
              """.format(name=name, gender=gender)
          cur.execute(sql)
      cur.execute("END") # cur.execute("COMMIT")
    except Exception as e:
      cur.execute("ROLLBACK") 

# Open Weathermap API: 서울 7일 평균/최소/최대 온도 읽어오기

In [ ]:
import requests, json

url = "https://api.openweathermap.org/data/2.5/onecall?lat=37.34&lon=126.57&exclude=current,minutely,hourly,alerts&appid=2528f873d1a0d14803ef7394abaf43b4"
response = requests.get(url)
data = json.loads(response.text) # data = response.json()

# 서울의 위도 경도 참고 정보 -> http://www.seoul.climatemps.com/map.php

In [ ]:
daily = data["daily"]

In [ ]:
from datetime import *

"""
 멱등성: 1번 실행하나 10번 연달아 실행하나 같은 레코드들을 갖고 있어야 한다
CREATE TABLE keeyong.seoul_temp (
    date DATE PRIMARY KEY,
    day float,
    min float,
    max float,
    created timestamp DEFAULT current_date()
);

ROW_NUMBER() OVER (PARTITION BY date ORDER BY created DESC) nn
"""

for entry in daily:
    dt = datetime.fromtimestamp(entry["dt"]).strftime('%Y-%m-%d') #날짜
    day = entry["temp"]["day"] #평균온도
    min = entry["temp"]["min"] #최소온도 
    max = entry["temp"]["max"] #최대온도
    print(f"{dt:11}{day:7}{min:7}{max:7}")

2021-10-16  285.36  280.4 292.23
2021-10-17  281.35 278.42 283.98
2021-10-18  285.06 282.88 288.69
2021-10-19  285.04 284.28 288.94
2021-10-20  284.11  282.7 285.77
2021-10-21  285.23  283.7 287.19
2021-10-22  286.73 285.53 288.55
2021-10-23  287.38 285.47 288.93
